

![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/healthcare_jsl/NER_CLINICAL_TRIALS_ABSTRACT.ipynb)




# `Extract Entities in Clinical Trial Abstracts`

To run this yourself, you will need to upload your license keys to the notebook. Just Run The Cell Below in order to do that. Also You can open the file explorer on the left side of the screen and upload `license_keys.json` to the folder that opens.
Otherwise, you can look at the example outputs at the bottom of the notebook.



# 🔎 Colab Setup

## Import license keys

In [ ]:
# Install the johnsnowlabs library to access Spark-OCR and Spark-NLP for Healthcare, Finance, and Legal.
! pip install johnsnowlabs

In [ ]:
from google.colab import files
print("Please Upload your John Snow Labs License using the button below")
license_keys = files.upload()

In [ ]:
from johnsnowlabs import *

# After uploading your license run this to install all licensed Python Wheels and pre-download Jars the Spark Session JVM
# Make sure to restart your notebook afterwards for changes to take effect

jsl.install()

## 2. Start Session

In [ ]:
from johnsnowlabs import *
# Automatically load license data and start a session with all jars user has access to
spark = jsl.start()

In [ ]:
spark

# 🔎 Model:

## ***`ner_clinical_trials_abstracts`***

🔎 It extracts relevant entities from clinical trial abstracts. It uses a simplified version of the ontology specified by Sanchez Graillet, O., et al. in order to extract concepts related to trial design, diseases, drugs, population, statistics and publication.

🔎 `Age`, `AllocationRatio`, `Author`, `BioAndMedicalUnit`, `CTAnalysisApproach`, `CTDesign`, `Confidence`, `Country`, `DisorderOrSyndrome`, `DoseValue`, `Drug`, `DrugTime`, `Duration`, `Journal`, `NumberPatients`, `PMID`, `PValue`, `PercentagePatients`, `PublicationYear`, `TimePoint`, `Value`

🔗 For more information about the model >>> [ner_clinical_trials_abstracts](https://nlp.johnsnowlabs.com/2022/06/22/ner_clinical_trials_abstracts_en_3_0.html)


# 🔎 Sample Text

In [7]:
sample_text = """Efficacy and safety of vildagliptin in patients with type 2 diabetes mellitus inadequately controlled with dual combination of metformin and sulphonylurea . This study assessed the efficacy and safety of vildagliptin as add - on therapy to metformin plus glimepiride combination in patients with type 2 diabetes mellitus ( T2DM ) who had inadequate glycaemic control . A multicentre , double - blind , placebo - controlled study randomized patients to receive treatment with vildagliptin 50   mg bid or placebo for 24   weeks ."""

In [16]:
df = spark.createDataFrame([[sample_text]]).toDF("text")
df.show(truncate=60)

+------------------------------------------------------------+
|                                                        text|
+------------------------------------------------------------+
|Efficacy and safety of vildagliptin in patients with type...|
+------------------------------------------------------------+



# 🔎 Define Spark NLP pipeline

In [9]:
document_assembler = nlp.DocumentAssembler() \
    .setInputCol('text')\
    .setOutputCol('document')

sentence_detector = nlp.SentenceDetector() \
    .setInputCols(['document'])\
    .setOutputCol('sentence')

tokenizer = nlp.Tokenizer()\
    .setInputCols(['sentence']) \
    .setOutputCol('token')

word_embeddings = nlp.WordEmbeddingsModel.pretrained('embeddings_clinical', 'en', 'clinical/models') \
    .setInputCols(['sentence', 'token']) \
    .setOutputCol('embeddings')

clinical_ner = medical.NerModel.pretrained("ner_clinical_trials_abstracts", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"])\
    .setOutputCol("ner")

ner_converter = nlp.NerConverter()\
    .setInputCols(['sentence', 'token', 'ner']) \
    .setOutputCol('ner_chunk')

nlp_pipeline = Pipeline(stages=[
    document_assembler, 
    sentence_detector,
    tokenizer,
    word_embeddings,
    clinical_ner,
    ner_converter
    ])


embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_clinical_trials_abstracts download started this may take some time.
[OK!]


# 🔎 Use the pipeline to create outputs

In [13]:
result = nlp_pipeline.fit(df).transform(df)

In [14]:
result.select(F.explode(F.arrays_zip("ner_chunk.result", "ner_chunk.metadata")).alias("cols"))\
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']['entity']").alias("ner_label")).show(truncate=False)

+------------------------+------------------+
|chunk                   |ner_label         |
+------------------------+------------------+
|vildagliptin            |Drug              |
|type 2 diabetes mellitus|DisorderOrSyndrome|
|metformin               |Drug              |
|sulphonylurea           |Drug              |
|vildagliptin            |Drug              |
|metformin               |Drug              |
|glimepiride             |Drug              |
|type 2 diabetes mellitus|DisorderOrSyndrome|
|T2DM                    |DisorderOrSyndrome|
|multicentre             |CTDesign          |
|double - blind          |CTDesign          |
|placebo                 |Drug              |
|randomized              |CTDesign          |
|vildagliptin            |Drug              |
|50                      |DoseValue         |
|mg                      |BioAndMedicalUnit |
|bid                     |DrugTime          |
|placebo                 |Drug              |
|24   weeks              |Duration

# 🔎 Visualize results

In [15]:
from sparknlp_display import NerVisualizer

NerVisualizer().display(
    result = result.collect()[0],
    label_col = 'ner_chunk',
    document_col = 'document'
)